In [45]:
import os
from pdfminer.high_level import extract_text
import nltk
from collections import Counter
import pandas as pd
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
import re
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import pos_tag
from collections import defaultdict
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from collections import Counter
import math
import numpy as np
from nltk import pos_tag

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ishathukral/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/ishathukral/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ishathukral/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
def convert_pdf_to_txt(source_folder, output_folder):

    for filename in os.listdir(source_folder):
        if filename.endswith(".pdf"):
            file_path = os.path.join(source_folder, filename)
            text = extract_text(file_path)
            output_file_path = os.path.join(output_folder, filename.replace('.pdf', '.txt'))
            with open(output_file_path, 'w', encoding='utf-8') as f:
                f.write(text)
            print(f"Converted {filename} to text and saved to {output_file_path}")

source_folder = '/Users/ishathukral/Downloads/DS5001project/sikh_data'
output_folder = '/Users/ishathukral/Downloads/DS5001project/sikh_text'

convert_pdf_to_txt(source_folder, output_folder)


Converted Siri Guru Granth .pdf to text and saved to /Users/ishathukral/Downloads/DS5001project/sikh_text/Siri Guru Granth .txt


In [16]:

def preprocess_text(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

def tokenize_and_count(text):
    tokens = word_tokenize(text)
    return len(tokens)

def find_sections(text):
    pattern = re.compile(r'\bRAAG\b.*?(?=\bRAAG\b|\Z)', re.DOTALL | re.IGNORECASE)
    return len(re.findall(pattern, text))

source_file_path = '/Users/ishathukral/Downloads/DS5001project/sikh_text/SiriGuruGranth.txt'

text = preprocess_text(source_file_path)
num_tokens = tokenize_and_count(text)
num_sections = find_sections(text)  

LIB = pd.DataFrame({
    'source_file_path': [source_file_path],
    'author': ['Multiple Gurus and Saints'],
    'title': ['Guru Granth Sahib'],
    'date': ['1607'],
    'length': [len(text)],
    'num_tokens': [num_tokens],
    'num_sections': [num_sections]  
})

LIB.index = [1]  #  this is the only document
LIB.index.name = 'book_id'


LIB


,source_file_path,author,title,date,length,num_tokens,num_sections
book_id,,,,,,,
1,/Users/ishathukral/Downloads/DS5001project/sik...,Multiple Gurus and Saints,Guru Granth Sahib,1607,5051999,1016165,370


In [22]:
LIB.to_csv('/Users/ishathukral/Downloads/DS5001project/output/LIB.csv', sep='|')

In [55]:
def preprocess_text(source_file_path):
    with open(source_file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    sections = re.split(r'RAAG \w+', text)
    corpus_data = []
    doc_id = 0
    for section in sections:
        if section.strip(): 
            doc_id += 1  
            for para_id, paragraph in enumerate(section.split('\n')):
                for sent_id, sentence in enumerate(sent_tokenize(paragraph)):
                    for token_id, token in enumerate(word_tokenize(sentence)):
                        pos = pos_tag([token])[0][1]  # Get POS tag for each token
                        corpus_data.append((doc_id, para_id, sent_id, token_id, token, token.lower(), pos))
                    
    columns = ['doc_id', 'para_id', 'sent_id', 'token_id', 'token_str', 'term_str', 'pos']
    return pd.DataFrame(corpus_data, columns=columns)

source_file_path = '/Users/ishathukral/Downloads/DS5001project/sikh_text/SiriGuruGranth.txt'
CORPUS = preprocess_text(source_file_path)


CORPUS

,doc_id,para_id,sent_id,token_id,token_str,term_str,pos
0,1,0,0,0,ÓÓÓÓÓÓÓÓÓÓÓ,óóóóóóóóóóó,NN
1,1,0,0,1,1,1,CD
2,1,0,0,2,ÓÓÓÓÓÓÓÓÓ,óóóóóóóóó,NN
3,1,2,0,0,ONE,one,CD
4,1,2,0,1,UNIVERSAL,universal,NN
...,...,...,...,...,...,...,...
1015766,208,60,1,0,||,||,NN
1015767,208,60,1,1,1,1,CD
1015768,208,60,1,2,||,||,NN
1015769,208,60,1,3,1,1,CD


In [50]:
CORPUS.to_csv('/Users/ishathukral/Downloads/DS5001project/output/CORPUS.csv', sep='|')

In [52]:

CORPUS = CORPUS[~CORPUS['term_str'].isin(['óóóóóóóóóóó', '||'])]  # Removing specific unwanted tokens

# Remove stopwords
stop_words = set(stopwords.words('english'))
CORPUS = CORPUS[~CORPUS['term_str'].isin(stop_words)]

VOCAB = CORPUS['term_str'].value_counts().reset_index()
VOCAB.columns = ['term_str', 'n']

VOCAB['df'] = CORPUS.groupby('term_str')['doc_id'].nunique().reset_index(drop=True)

N_docs = CORPUS['doc_id'].nunique()

VOCAB['idf'] = np.log((N_docs + 1) / (VOCAB['df'] + 1)) + 1

VOCAB['dfidf'] = VOCAB['df'] * VOCAB['idf']

stemmer = PorterStemmer()
VOCAB['porter_stem'] = VOCAB['term_str'].apply(lambda x: stemmer.stem(x))

# Calculate max POS for each term
def get_max_pos(term):
    pos_list = CORPUS[CORPUS['term_str'] == term]['pos']
    return pos_list.mode()[0] if not pos_list.empty else 'NN'  # Default to 'NN' if no mode found

VOCAB['max_pos'] = VOCAB['term_str'].apply(get_max_pos)
VOCAB['stop'] = VOCAB['term_str'].apply(lambda x: x in stop_words)

VOCAB.sort_values(by='dfidf', ascending=False).head(20)



,term_str,n,df,idf,dfidf,porter_stem,max_pos,stop
1465,respect,36,208,1.000000,208.000000,respect,NN,False
5,one,6479,208,1.000000,208.000000,one,CD,False
18,naam,2696,208,1.000000,208.000000,naam,NN,False
17,shall,2709,208,1.000000,208.000000,shall,MD,False
567,all-pervading,137,207,1.004796,207.992808,all-pervad,NN,False
7708,unconcerned,1,205,1.014458,207.963907,unconcern,JJ,False
6856,leafy,2,204,1.019324,207.942152,leafi,NN,False
1466,weight,36,202,1.029128,207.883911,weight,NN,False
10954,ever-young,1,199,1.044017,207.759360,ever-young,NN,False
5363,atonement,3,198,1.049029,207.707827,aton,NN,False


In [53]:
VOCAB.to_csv('/Users/ishathukral/Downloads/DS5001project/output/VOCAB.csv', sep='|', index=False)